In [1]:
import sys

print(sys.version)

3.8.0 (default, Nov  6 2019, 15:49:01) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import BertModel, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange

from keras.preprocessing.sequence import pad_sequences

from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
import csv

In [4]:
data = pd.read_csv("pdb_data_no_dups.csv")
data


,structureId,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,1.78,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0
1,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,NaN,NaN,2.00,38.45,NaN,NaN,1995.0
2,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,NaN,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0
3,102D,DNA,X-RAY DIFFRACTION,DNA,24,2.20,7637.17,"VAPOR DIFFUSION, SITTING DROP",277.0,2.28,46.06,"pH 7.00, VAPOR DIFFUSION, SITTING DROP, temper...",7.0,1995.0
4,102L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,165,1.74,18926.61,NaN,NaN,2.75,55.28,NaN,NaN,1993.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141396,9RUB,LYASE(CARBON-CARBON),X-RAY DIFFRACTION,Protein,932,2.60,101838.68,NaN,NaN,2.38,48.29,NaN,NaN,1991.0
141397,9TNA,T-RNA,X-RAY DIFFRACTION,NaN,0,NaN,24244.34,NaN,NaN,3.17,61.18,NaN,NaN,1986.0
141398,9WGA,LECTIN (AGGLUTININ),X-RAY DIFFRACTION,Protein,342,1.80,34270.22,NaN,NaN,2.50,50.76,NaN,NaN,1990.0
141399,9XIA,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),X-RAY DIFFRACTION,Protein,388,1.90,43542.29,NaN,NaN,2.79,55.93,NaN,NaN,1989.0


In [5]:
data.drop(data.iloc[:, 4:], inplace = True, axis = 1)
data = data.drop(["structureId", "experimentalTechnique"] , axis = 1)


In [6]:
data.info()
data = data.dropna()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141401 entries, 0 to 141400
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   classification     141399 non-null  object
 1   macromoleculeType  137636 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


,classification,macromoleculeType
0,DNA-RNA HYBRID,DNA/RNA Hybrid
1,DNA,DNA
2,OXYGEN TRANSPORT,Protein
3,DNA,DNA
4,HYDROLASE(O-GLYCOSYL),Protein
...,...,...
141395,HYDROLASE (PHOSPHORIC DIESTER),Protein
141396,LYASE(CARBON-CARBON),Protein
141398,LECTIN (AGGLUTININ),Protein
141399,ISOMERASE(INTRAMOLECULAR OXIDOREDUCTASE),Protein


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137634 entries, 0 to 141400
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   classification     137634 non-null  object
 1   macromoleculeType  137634 non-null  object
dtypes: object(2)
memory usage: 3.2+ MB


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [10]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in data["classification"]]
tokenized_labels = [tokenizer.tokenize(sent) for sent in data["macromoleculeType"]]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
labels = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_labels]

In [11]:
MAX_LEN = 32


input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


In [12]:
MAX_LEN_2 = 1

labels = pad_sequences(labels, maxlen=MAX_LEN_2, dtype="long", truncating="post", padding="post")

In [13]:
labels

array([[6064],
       [6064],
       [5250],
       ...,
       [5250],
       [5250],
       [5250]])

In [50]:
asd = pd.DataFrame(input_ids)

In [51]:
asd

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,6064,1011,12987,8893,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6064,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7722,3665,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6064,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18479,25002,1006,1051,1011,1043,2135,13186,8516,1007,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137629,18479,25002,1006,6887,2891,8458,29180,8289,3334,1007,...,0,0,0,0,0,0,0,0,0,0
137630,1048,16303,2063,1006,6351,1011,6351,1007,0,0,...,0,0,0,0,0,0,0,0,0,0
137631,3393,6593,2378,1006,12943,23296,21823,11483,1007,0,...,0,0,0,0,0,0,0,0,0,0
137632,11163,5017,11022,1006,26721,5302,2571,15431,23060,13820,...,0,0,0,0,0,0,0,0,0,0


In [55]:
asd1 = asd.iloc[:135768]

In [58]:
input_ids = asd1.to_numpy()

In [59]:
a12 = pd.DataFrame(labels)
a13 = a12.replace([5250],0)
a14 = a13.replace([12987], 1)
xd = a14.replace([6064], 2)

In [60]:
ad = xd.to_numpy()


In [61]:
new_labels = np.delete(ad, np.where(ad == 2))

    

In [62]:
new_labels

array([0, 0, 0, ..., 0, 0, 0])

In [63]:
X_train, X_test,y_train,y_test = train_test_split(input_ids, new_labels, test_size = 0.2, random_state = 42)



In [64]:
a = pd.DataFrame(X_train)
a.describe()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
count,108614.000000,108614.000000,108614.000000,108614.000000,108614.000000,108614.000000,108614.000000,108614.000000,108614.000000,108614.000000,...,108614.0,108614.0,108614.0,108614.0,108614.0,108614.0,108614.0,108614.0,108614.0,108614.0
mean,11875.193815,11535.633196,3080.948561,3343.560766,3560.458385,1675.744039,509.957206,430.967932,175.903723,133.170834,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,7224.735784,8282.720111,5791.967819,6239.134547,7154.777548,5656.774277,3048.688436,2692.558513,1555.290664,1382.487129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1005.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,4651.000000,5250.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,11311.000000,11022.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,18479.000000,16303.000000,5250.000000,3665.000000,1013.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,29610.000000,29578.000000,29598.000000,29598.000000,29358.000000,29336.000000,29598.000000,28517.000000,29090.000000,29206.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
b = pd.DataFrame(y_train)
b[:32329]

,0
0,0
1,0
2,0
3,0
4,0
...,...
32324,0
32325,0
32326,0
32327,0


In [66]:
t = b.replace([5250],0)
t = t.replace([12987], 1)
t

,0
0,0
1,0
2,0
3,0
4,0
...,...
108609,0
108610,0
108611,0
108612,0


In [67]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108614 entries, 0 to 108613
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   0       108614 non-null  int64
dtypes: int64(1)
memory usage: 848.7 KB


In [68]:
t.describe()

,0
count,108614.000000
mean,0.009686
std,0.097939
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [69]:
print(t.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108614 entries, 0 to 108613
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   0       108614 non-null  int64
dtypes: int64(1)
memory usage: 848.7 KB
None


In [70]:
t[0].unique()

array([0, 1])

In [71]:
ar = t[0].values

In [72]:
train_inputs = torch.tensor(X_train)
validation_inputs = torch.tensor(X_test)
train_labels = torch.tensor(y_train)
validation_labels = torch.tensor(y_test)

In [73]:
train_inputs

tensor([[11311,  2291,     0,  ...,     0,     0,     0],
        [18479, 25002,     0,  ...,     0,     0,     0],
        [ 5250,  8031,     0,  ...,     0,     0,     0],
        ...,
        [ 3665,  5250,     0,  ...,     0,     0,     0],
        [23060, 13820,  5596,  ...,     0,     0,     0],
        [ 4651, 11022,     0,  ...,     0,     0,     0]])

In [74]:
train_labels

tensor([0, 0, 0,  ..., 0, 0, 1])

In [75]:
validation_inputs

tensor([[13434,  5250,     0,  ...,     0,     0,     0],
        [29090,     0,     0,  ...,     0,     0,     0],
        [14828,  5250,     0,  ...,     0,     0,     0],
        ...,
        [23060, 13820,  5596,  ...,     0,     0,     0],
        [ 5250,  8031,     0,  ...,     0,     0,     0],
        [ 9706,  7361, 25950,  ...,     0,     0,     0]])

In [76]:
validation_labels

tensor([0, 0, 0,  ..., 0, 0, 0])

In [77]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [78]:
bert = AutoModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [79]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels= 2)

In [80]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay_rate': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay_rate': 0.0}]

In [81]:
optimizer = BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)

t_total value of -1 results in schedule not being applied


In [82]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [83]:
#t = [] 

# Store our loss and accuracy for plotting
#train_loss_set = []

# Number of training epochs 
#epochs = 2




In [85]:
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs 
epochs = 2

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    # Unpack the inputs from our dataloader
    b_input_ids, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
 

Epoch:   0%|          | 0/2 [01:45<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
 # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    # batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))